In [18]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


# OpenAI version

In [20]:
prompt = ["Write a hello world program in Python", "What is the fourth prime number?"]
response = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    temperature=0,
    max_tokens=64,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logprobs=1,
    echo=True,
    n=3,
    stop=["\"\"\""]
)

In [21]:
response

<OpenAIObject text_completion id=cmpl-70oqLp8fIFIJJi7vmG933qdV4OYiw at 0x7f9cd6561490> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "text_offset": [
          0,
          5,
          7,
          13,
          19,
          27,
          30,
          37,
          38,
          39,
          44,
          46,
          51,
          52,
          58,
          60
        ],
        "token_logprobs": [
          null,
          -4.210584,
          -8.784183,
          -0.14352785,
          -0.028228737,
          -0.062067956,
          -3.3281035,
          -0.0060410043,
          -0.00015875288,
          -0.009935659,
          -0.14525835,
          -0.0014810939,
          -0.09724497,
          -0.29126295,
          -0.000865787,
          -0.00016638411
        ],
        "tokens": [
          "Write",
          " a",
          " hello",
          " world",
          " program",
          " in",
          " Pytho

In [30]:
len(response['choices'])

6

In [27]:
[response['choices'][i]['text'] for i in range(len(response['choices']))]

['Write a hello world program in Python\n\nprint("Hello, World!")',
 'Write a hello world program in Python\n\nprint("Hello, World!")',
 'Write a hello world program in Python\n\nprint("Hello, World!")',
 'What is the fourth prime number?\n\nThe fourth prime number is 7.',
 'What is the fourth prime number?\n\nThe fourth prime number is 7.',
 'What is the fourth prime number?\n\nThe fourth prime number is 7.']

In [23]:
response

<OpenAIObject text_completion id=cmpl-6xWLbYVaGy8LGGq9x3gUpJEuadudR at 0x7fae9bf8f9c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "text_offset": [
          0,
          5,
          7,
          13,
          19,
          27,
          30,
          37,
          38,
          39,
          44,
          46,
          51,
          52,
          58,
          60
        ],
        "token_logprobs": [
          null,
          -4.2096,
          -8.782522,
          -0.14281924,
          -0.027957046,
          -0.060565412,
          -3.3293993,
          -0.0059190392,
          -0.00015839339,
          -0.009974006,
          -0.14499061,
          -0.0014710035,
          -0.09777169,
          -0.28776452,
          -0.0009090991,
          -0.00015732239
        ],
        "tokens": [
          "Write",
          " a",
          " hello",
          " world",
          " program",
          " in",
          " Python

In [19]:
from transformers import pipeline

In [16]:
input_text = "For Halloween Debby and her sister combined the candy they received. Debby had 32 pieces of candy while her sister had 42. If they ate 35 pieces the first night, how many pieces do they have left?"

In [22]:
pipe = pipeline(model="google/flan-t5-large", device=0)

In [26]:
out = pipe(input_text)

# HuggingFace version

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

dev = 'cuda'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large", device_map="auto")

In [4]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-large', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>

In [40]:
def do_predict(input_texts, model, tokenizer, n=1):
    """
    Takes in a list of text inputs and generates output text for each input using the model and tokenizer.

    Args:
        input_texts (List[str]): List of text inputs to be used as input to the model.
        model (transformers.PreTrainedModel): Model to be used for prediction.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer to be used for prediction.
        n (int, optional): Number of predictions to generate for each input. Defaults to 1.

    Returns:
        List[str]: List of generated text.
    """
    # Encode the input texts
    input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)

    # Generate output with the model
    outputs = model.generate(
        input_ids,
        max_length=30,
        num_return_sequences=n,
        do_sample=True,
        return_dict_in_generate=True,
        output_scores=True)

    # Get the generated tokens
    generated_tokens = outputs.sequences

    # Convert tokens to text
    generated_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Calculate log probabilities
    # logprobs = sum([torch.log_softmax(score, dim=-1).gather(1, token.view(-1, 1)).squeeze() for score, token in zip(outputs.scores, generated_tokens)])

    return generated_texts

In [41]:
do_predict(["Hello world program in Python", "What is the fourth prime number?"], model, tokenizer, n=3)

["Hello world program calls _input() and _output(). I didn'",
 'PHP program to calculate the index of an atom of hydrogen in a graph',
 '""" This page was last modified on 17 April 2016, at 09:02.',
 '230725',
 '111',
 '5']

In [31]:
input_text = "For Halloween Debby and her sister combined the candy they received. Debby had 32 pieces of candy while her sister had 42. If they ate 35 pieces the first night, how many pieces do they have left?"
preprompt_1 = ""
postprompt_1 = ""

preprompt_2 = ""
postprompt_2 = "Let's think step by step."

input_ids_1 = tokenizer(preprompt_1 + input_text + postprompt_1, return_tensors="pt").input_ids.to(dev)
input_ids_2 = tokenizer(preprompt_2 + input_text + postprompt_2, return_tensors="pt").input_ids.to(dev)

In [32]:
output_1 = model.generate(input_ids_1, max_length = 300, return_dict_in_generate=True, output_scores = True)
output_2 = model.generate(input_ids_2, max_length = 300, return_dict_in_generate=True, output_scores = True)

output_1

GreedySearchEncoderDecoderOutput(sequences=tensor([[    0,  9794,   969,    11,   160,  4806,  1204,     3,     9,   792,
            13,  3538,  1768,  6426,  3274,     3,  3539,  2161,    13, 11617,
             5,   621,  3182,  3097,  2161,     6,    79,    43,     3,  3539,
             3,    18,  3097,  3274,     3,  3539,  2161,    13, 11617,   646,
             5,    37,  1525,    10,     3,  3539,     5,     1]],
       device='cuda:0'), scores=(tensor([[-52.3152,  -7.8264, -13.2415,  ..., -52.2748, -52.1174, -52.3978]],
       device='cuda:0'), tensor([[-61.5653, -14.5178, -14.6825,  ..., -61.5489, -61.4221, -61.9239]],
       device='cuda:0'), tensor([[-51.8955,  -9.5711,  -9.0433,  ..., -52.0970, -51.9487, -52.1281]],
       device='cuda:0'), tensor([[-52.8280, -10.7344, -10.5026,  ..., -52.6193, -52.7292, -52.9024]],
       device='cuda:0'), tensor([[-59.7848, -12.9511, -13.3107,  ..., -59.5814, -59.6824, -59.9058]],
       device='cuda:0'), tensor([[-53.7566,  -9.9275, -1

In [13]:
answer_1 = tokenizer.decode(output_1['sequences'][0], skip_special_tokens=True)

print(answer_1)


Debby and her sister received a total of 32 + 42 = 66 pieces of candy. After eating 35 pieces, they have 66 - 35 = 66 pieces of candy left. The answer: 66.
